In [106]:
from collections import Counter
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
import json
import configparser
from TwitterAPI import TwitterAPI

In [107]:
def get_twitter(config_file):
    """ Read the config_file and construct an instance of TwitterAPI.
    Args:
      config_file ... A config file in ConfigParser format with Twitter credentials
    Returns:
      An instance of TwitterAPI.
    """
    config = configparser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter

twitter = get_twitter('twitter.cfg')
print('Established Twitter connection.')

Established Twitter connection.


In [108]:
# 1 - Read the screen names we want to build the network from.
def read_screen_names(filename):
    """
    Read a text file containing Twitter screen_names, one per line.

    Params:
        filename....Name of the file to read.
    Returns:
        A list of strings, one per screen_name, in the order they are listed
        in the file.

    Here's a doctest to confirm your implementation is correct.
    >>> read_screen_names('candidates.txt')
    ['DrJillStein', 'GovGaryJohnson', 'HillaryClinton', 'realDonaldTrump']
    """
    file = open(filename)
    r = sorted([l.strip() for l in file])
    file.close()
    return r

screen_names = read_screen_names('ethereum-accounts.txt')
print('Read screen names: %s' % screen_names)

Read screen names: ['VitalikButerin', 'binance', 'coinbase', 'consenSysAcad', 'ethereum', 'gavofyork', 'trufflesuite']


In [109]:
# I've provided the method below to handle Twitter's rate limiting.
def robust_request(twitter, resource, params, max_tries=5):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request
      params ..... A parameter dict for the request, e.g., to specify
                   parameters like screen_name or count.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)
        if request.status_code == 200 | request.status_code == 404:
            return request
        else:
            print('Got error %s \n with status code %s\nsleeping for 15 minutes.' % (request.text, request.status_code))
            sys.stderr.flush()
            time.sleep(61 * 15)

In [110]:
def robust_request_iterate(twitter, resource, params, max_pages=5):
    """ Function for managing pagination of results
    It will sequentially obtain all the pages using the cursor provided by Twittter.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request
      params ..... A parameter dict for the request, e.g., to specify
                   parameters like screen_name or count.
      max_pages .. The maximum number of pages to ask for.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    ### TODO:
    cursor = -1
    # Add cursor parameter to params:
    params['cursor'] = cursor
    print(params)
    results = []
    while True:
        response = robust_request(twitter, resource, params)
        results.extend(response)
        print(response)
        next_cursor = response.json()['next_cursor']
        print(next_cursor)
        params['cursor'] = next_cursor
        if (next_cursor == 0):
            break
    return results

In [111]:
# 2 - Get the user's twitter accounts
def get_users_by_screen_name(twitter, screen_names):
    """Retrieve the Twitter user objects for each screen_name.
    Params:
        twitter........The TwitterAPI object.
        screen_names...A list of strings, one per screen_name
    Returns:
        A list of dicts, one per user, containing all the user information
        (e.g., screen_name, id, location, etc)

    >>> twitter = get_twitter()
    >>> users = get_users(twitter, ['twitterapi', 'twitter'])
    >>> [u['id'] for u in users]
    [6253282, 783214]
    """
    results = []
    for s in screen_names:
        # Erase traling @ of screen names:
        
        results.extend(robust_request(twitter, "users/lookup", {'screen_name': s}))
    return results

users = sorted(get_users_by_screen_name(twitter, screen_names), key=lambda x: x['screen_name'])
print('found %d users with screen_names %s' %
    (len(users), str([u['screen_name'] for u in users])))
# print(users)

found 7 users with screen_names ['ConsenSysAcad', 'VitalikButerin', 'binance', 'coinbase', 'ethereum', 'gavofyork', 'trufflesuite']


In [112]:
#print(users[4])
print(users[4][list(users[4])[0]])
print(users[1]['screen_name'])

2312333412
VitalikButerin


In [148]:
# 3 - Get the user's each of this user follows (their "friends")
def get_friends(twitter, screen_name):
    """ Return a list of Twitter IDs for users that this person follows, up to 5000.
    See https://dev.twitter.com/rest/reference/get/friends/ids

    Args:
        twitter.......The TwitterAPI object
        screen_name... a string of a Twitter screen name
    Returns:
        A list of ints, one per friend ID, sorted in ascending order.

    Note: If a user follows more than 5000 accounts, we will limit ourselves to
    the first 5000 accounts returned.

    In this test case, I return the first 5 accounts that I follow.
    >>> twitter = get_twitter()
    >>> get_friends(twitter, 'aronwc')[:5]
    [695023, 1697081, 8381682, 10204352, 11669522]
    """
    results = []
    print("Inside: Requesting friends for screen_name %s" % screen_name)
    request = robust_request(twitter, "friends/ids", 
                              {'screen_name': screen_name, 'count':5000})
    if (request.status_code != 404):
        return sorted(request.json()['ids'])
    else:
        return str(request.status_code)

#print("Looking for friends of @%s" % users[2]['screen_name'])
#binance_friends = get_friends(twitter, users[2]['screen_name'])
#print('Complete response')
#print(binance_friends)

In [154]:
# Obtain all friends
def add_all_friends(twitter, users):
    """ Get the list of accounts each user follows.

    Store the result in each user's dict using a new key called 'friends'.

    Args:
        twitter...The TwitterAPI object.
        users.....The list of user dicts.
    Returns:
        Nothing

    >>> twitter = get_twitter()
    >>> users = [{'screen_name': 'aronwc'}]
    >>> add_all_friends(twitter, users)
    >>> users[0]['friends'][:5]
    [695023, 1697081, 8381682, 10204352, 11669522]
    """
    print("Requesting friends for a total of %s users" % len(users))
    for u in users:
        print("Outside: Requesting friends for screen_name %s" % u['screen_name'])
        # Make the requst only if the user is not protected, else store friends as an empty list
        if u['protected'] != True:
            response = get_friends(twitter, u['screen_name'])
            if response == "404":
                u['friends'] = []
            else:
                u['friends'] = response
        else:
            #
            u['friends'] = []
        
#add_all_friends(twitter, users)

In [155]:
def print_num_friends(users):
    """Print the number of friends per candidate, sorted by candidate name.
    See Log.txt for an 
    example.
    Args:
        users....The list of user dicts.
    Returns:
        Nothing
    """
    print('\n'.join('%s %d' % (u['screen_name'], len(u['friends'])) for u in users))
        
print('Friends per candidate:')
print_num_friends(users)

Friends per candidate:
ConsenSysAcad 233
VitalikButerin 135
binance 171
coinbase 11
ethereum 0
gavofyork 96
trufflesuite 229


In [156]:
def count_friends(users):
    """ Count how often each friend is followed.
    Args:
        users: a list of user dicts
    Returns:
        a Counter object mapping each friend to the number of candidates who follow them.
        Counter documentation: https://docs.python.org/dev/library/collections.html#collections.Counter

    In this example, friend '2' is followed by three different users.
    >>> c = count_friends([{'friends': [1,2]}, {'friends': [2,3]}, {'friends': [2,3]}])
    >>> c.most_common()
    [(2, 3), (3, 2), (1, 1)]
    """
    ###TODO
    result = Counter()
    for u in users:
        result.update(u['friends'])
    return result

friend_counts = count_friends(users)
print('Most common friends:\n%s' % str(friend_counts.most_common(5)))

Most common friends:
[(315991624, 4), (282365019, 3), (2362854624, 3), (14182218, 3), (44196397, 3)]


In [157]:
# Who follows this person?
# Pick the first 2.000 people that follow this person.
# 3 - Get the user's each of this user follows (their "friends")
def get_followers(twitter, screen_name):
    """ Return a list of Twitter IDs for users that follow this person, up to 2000.
    See https://developer.twitter.com/en/docs/accounts-and-users/follow-search-get-users/api-reference/get-followers-list

    Args:
        twitter.......The TwitterAPI object
        screen_name... a string of a Twitter screen name
    Returns:
        A list of ints, one per friend ID, sorted in ascending order.

    Note: If a user has more than 5000 followers, we will limit ourselves to
    the first 5000 accounts returned.

    In this test case, I return the first 5 accounts that I follow.
    >>> twitter = get_twitter()
    >>> get_friends(twitter, 'aronwc')[:5]
    [695023, 1697081, 8381682, 10204352, 11669522]
    """
    print("Inside: Requesting followers for screen_name %s" % screen_name)
    request = robust_request(twitter, "followers/ids", 
                              {'screen_name': screen_name, 'count':5000})
    if (request.status_code != 404):
        return sorted(request.json()['ids'])
    else:
        return str(request.status_code)

In [158]:
# Obtain all followers for each user
def add_all_followers(twitter, users):
    """ Get all the followers a user has.

    Store the result in each user's dict using a new key called 'followers'.

    Args:
        twitter...The TwitterAPI object.
        users.....The list of user dicts.
    Returns:
        Nothing

    >>> twitter = get_twitter()
    >>> users = [{'screen_name': 'aronwc'}]
    >>> add_all_followers(twitter, users)
    >>> users[0]['followers'][:5]
    [695023, 1697081, 8381682, 10204352, 11669522]
    """
    for u in users:
        print("Outside: Requesting followers for screen_name %s" % u['screen_name'])
        if u['protected'] != True:
            response = get_followers(twitter, u['screen_name'])
            if response == "404":
                u['followers'] = []
            else:
                u['followers'] = response
        else:
            u['followers'] = []
        
add_all_followers(twitter, users)

Outside: Requesting followers for screen_name ConsenSysAcad
Inside: Requesting followers for screen_name ConsenSysAcad
Outside: Requesting followers for screen_name VitalikButerin
Inside: Requesting followers for screen_name VitalikButerin
Outside: Requesting followers for screen_name binance
Inside: Requesting followers for screen_name binance
Outside: Requesting followers for screen_name coinbase
Inside: Requesting followers for screen_name coinbase
Outside: Requesting followers for screen_name ethereum
Inside: Requesting followers for screen_name ethereum
Outside: Requesting followers for screen_name gavofyork
Inside: Requesting followers for screen_name gavofyork
Outside: Requesting followers for screen_name trufflesuite
Inside: Requesting followers for screen_name trufflesuite


In [171]:
def print_num_followers(users):
    """Print the number of followers per candidate, sorted by candidate name.
    Args:
        users....The list of user dicts.
    Returns:
        Nothing
    """
    print('\n'.join('%s %d' % (u['screen_name'], len(u['followers'])) for u in users))
        
print('Followers per candidate:')
print_num_followers(users)

Followers per candidate:
ConsenSysAcad 2368
VitalikButerin 5000
binance 5000
coinbase 5000
ethereum 5000
gavofyork 5000
trufflesuite 5000


In [172]:
def count_friends_and_followers(users):
    """ Count how often each user is followed or follows the defined users.
    Args:
        users: a list of user dicts
    Returns:
        a Counter object mapping each friend to the number of candidates who follow them.
        Counter documentation: https://docs.python.org/dev/library/collections.html#collections.Counter

    In this example, friend '2' is followed by three different users.
    >>> c = count_friends([{'friends': [1,2]}, {'friends': [2,3]}, {'friends': [2,3]}])
    >>> c.most_common()
    [(2, 3), (3, 2), (1, 1)]
    """
    ###TODO
    result = Counter()
    for u in users:
        result.update(u['friends'])
        result.update(u['followers'])
    return result

friend_and_followers_counts = count_friends_and_followers(users)
most_common_100 = friend_and_followers_counts.most_common(100)
print('Most common friends and followers:\n%s' % str(friend_and_followers_counts.most_common(5)))

Most common friends and followers:
[(1102918561339711489, 8), (2744100241, 5), (3879800793, 5), (913131503667859456, 5), (1075046092612530177, 5)]


In [173]:
# print(type(most_common_100[0]))
# print(most_common_100[0][0])
# Store in a list all the id's of the 100 most common users
most_common_100_ids = [user[0] for user in most_common_100]
# print(most_common_100_ids)

In [177]:
# 4 - For each of the most common 100 users, obtain the user object and append it to the list of users' dict.
def get_users_by_ids(twitter, ids):
    """Retrieve the Twitter user objects for each id.
    Params:
        twitter........The TwitterAPI object.
        ids............A list of strings, one per id
    Returns:
        A list of dicts, one per user, containing all the user information
        (e.g., screen_name, id, location, etc)

    >>> twitter = get_twitter()
    >>> users = get_users(twitter, ['twitterapi', 'twitter'])
    >>> [u['id'] for u in users]
    [6253282, 783214]
    """
    results = robust_request(twitter, "users/lookup", {'user_id': ids})
    return sorted(results.json(), key=lambda x: x['screen_name'])

new_users = get_users_by_ids(twitter, most_common_100_ids)
print('found %d users with screen_names %s' %
    (len(new_users), str([u['screen_name'] for u in new_users])))

found 100 users with screen_names ['AdheBro', 'AdilHaris6', 'Albert81262341', 'Anvarkuliev1', 'BauThang1983', 'BradyMck_', 'CE150d', 'Chasethesun5', 'Christo60012017', 'CoinmetroMemes', 'ConsenSysIre', 'ConsensysDesign', 'ConsensysNews', 'CryptoKeen2', 'CryptoNotifyApp', 'CryptoistBTC', 'DacebookAlan', 'Dimon51311049', 'ErikHolec', 'EtherealSummit', 'FEhrsam', 'FalouDarke', 'HIEUBUITRAVEL', 'HardforkedR', 'JD_CDMX', 'JesterInfinite', 'KinsMusafau', 'MandegarInvest', 'ManuelD64140759', 'MarkBeylin', 'MarkCur56286904', 'MatthieuLux', 'Mc29806081', 'MelBasson', 'MikeWise07', 'Mustafa72211070', 'Narmi1986', 'Nate30078132', 'NowPayments', 'Oscarsim22', 'PW78200124', 'Pawande21956316', 'PhilAnagnos', 'RoscoKalis', 'Scottg_olson', 'SmartMatter_io', 'Str0kierKJ', 'TrudelAndrew', 'TrumpMiller', 'TvKinetic', 'YEE999999999', 'ZAXTONOFFICIAL', 'adeshzamani', 'amandagutterman', 'ameensol', 'andyjang93', 'angella19191', 'anthonyramos88x', 'asif_balkhi', 'atomix_official', 'blacque64', 'brahimAyka11'

In [179]:
add_all_friends(twitter, new_users)
add_all_followers(twitter, new_users)

Requesting friends for a total of 100 users
Outside: Requesting friends for screen_name AdheBro
Inside: Requesting friends for screen_name AdheBro
Got error {"errors":[{"message":"Rate limit exceeded","code":88}]} 
 with status code 429
sleeping for 15 minutes.
Outside: Requesting friends for screen_name AdilHaris6
Inside: Requesting friends for screen_name AdilHaris6
Outside: Requesting friends for screen_name Albert81262341
Inside: Requesting friends for screen_name Albert81262341
Outside: Requesting friends for screen_name Anvarkuliev1
Inside: Requesting friends for screen_name Anvarkuliev1
Outside: Requesting friends for screen_name BauThang1983
Inside: Requesting friends for screen_name BauThang1983
Outside: Requesting friends for screen_name BradyMck_
Inside: Requesting friends for screen_name BradyMck_
Outside: Requesting friends for screen_name CE150d
Inside: Requesting friends for screen_name CE150d
Outside: Requesting friends for screen_name Chasethesun5
Inside: Requesting fr

Outside: Requesting friends for screen_name hamawby
Inside: Requesting friends for screen_name hamawby
Outside: Requesting friends for screen_name hetman_adrian
Inside: Requesting friends for screen_name hetman_adrian
Outside: Requesting friends for screen_name hussainmoazzam9
Inside: Requesting friends for screen_name hussainmoazzam9
Outside: Requesting friends for screen_name iamtusharkhade
Inside: Requesting friends for screen_name iamtusharkhade
Outside: Requesting friends for screen_name incognito13337
Outside: Requesting friends for screen_name iranexchangeio
Inside: Requesting friends for screen_name iranexchangeio
Outside: Requesting friends for screen_name joey_kalchuk
Inside: Requesting friends for screen_name joey_kalchuk
Outside: Requesting friends for screen_name klinik_ua
Inside: Requesting friends for screen_name klinik_ua
Outside: Requesting friends for screen_name knevitz5
Inside: Requesting friends for screen_name knevitz5
Outside: Requesting friends for screen_name l

Outside: Requesting followers for screen_name Scottg_olson
Inside: Requesting followers for screen_name Scottg_olson
Outside: Requesting followers for screen_name SmartMatter_io
Inside: Requesting followers for screen_name SmartMatter_io
Outside: Requesting followers for screen_name Str0kierKJ
Inside: Requesting followers for screen_name Str0kierKJ
Outside: Requesting followers for screen_name TrudelAndrew
Inside: Requesting followers for screen_name TrudelAndrew
Got error {"errors":[{"message":"Rate limit exceeded","code":88}]} 
 with status code 429
sleeping for 15 minutes.
Outside: Requesting followers for screen_name TrumpMiller
Inside: Requesting followers for screen_name TrumpMiller
Outside: Requesting followers for screen_name TvKinetic
Inside: Requesting followers for screen_name TvKinetic
Outside: Requesting followers for screen_name YEE999999999
Inside: Requesting followers for screen_name YEE999999999
Outside: Requesting followers for screen_name ZAXTONOFFICIAL
Inside: Reque

In [190]:
type(new_users)
a = users + new_users
users_total = users + new_users

In [192]:
# Save users list of dicts to a users.json file
with open('users.json', 'w+') as fout:
    json.dump(users_total, fout)

In [193]:
with open('users.json', 'r') as fin:
    users_read = json.load(fin)

In [194]:
len(users_read)

107